In [3]:
%%writefile gpu_memory_optimization.cu

#include <cuda_runtime.h>              // Основные функции CUDA
#include <device_launch_parameters.h>  // Параметры запуска kernel

#include <iostream>    // Ввод-вывод
#include <vector>      // Контейнер vector
#include <random>      // Генерация случайных чисел

#define THREADS 256     // Количество потоков в одном CUDA-блоке

// =====================================================
// Генерация массива случайных чисел (на CPU)
// Используется для подготовки входных данных
// =====================================================
void generateArray(std::vector<int>& a) {
    std::mt19937 gen(42);                          // Фиксированный seed
    std::uniform_int_distribution<> dist(0, 100); // Диапазон значений
    for (auto& x : a)
        x = dist(gen);
}

// =====================================================
// REDUCTION 1: ТОЛЬКО ГЛОБАЛЬНАЯ ПАМЯТЬ
// Каждый поток добавляет свой элемент через atomicAdd
// =====================================================
__global__ void reductionGlobal(int* data, int* result, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверка выхода за границы массива
    if (idx < n) {
        // Атомарное сложение в глобальной памяти
        atomicAdd(result, data[idx]);
    }
}

// =====================================================
// REDUCTION 2: ГЛОБАЛЬНАЯ + РАЗДЕЛЯЕМАЯ ПАМЯТЬ
// Редукция сначала выполняется внутри блока
// =====================================================
__global__ void reductionShared(int* data, int* result, int n) {
    // Разделяемая память для текущего блока
    __shared__ int sdata[THREADS];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + tid;

    // Загрузка данных из глобальной памяти в shared memory
    sdata[tid] = (idx < n) ? data[idx] : 0;
    __syncthreads();

    // Параллельная редукция внутри блока
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (tid < stride)
            sdata[tid] += sdata[tid + stride];
        __syncthreads();
    }

    // Запись частичной суммы блока в глобальную память
    if (tid == 0)
        atomicAdd(result, sdata[0]);
}

// =====================================================
// BUBBLE SORT ДЛЯ ПОДМАССИВОВ
// Используются локальные переменные потоков
// =====================================================
__global__ void bubbleSubarray(int* data, int chunk) {
    // Начальный индекс подмассива для данного блока
    int start = blockIdx.x * chunk;

    // Классическая сортировка пузырьком
    for (int i = 0; i < chunk; i++) {
        for (int j = start; j < start + chunk - 1; j++) {
            int a = data[j];
            int b = data[j + 1];

            if (a > b) {
                data[j]     = b;
                data[j + 1] = a;
            }
        }
    }
}

// =====================================================
// СЛИЯНИЕ ОТСОРТИРОВАННЫХ ПОДМАССИВОВ
// Используется разделяемая память
// =====================================================
__global__ void mergeShared(int* input, int* output, int width, int n) {
    // Динамически выделяемая shared memory
    extern __shared__ int s[];

    int tid   = threadIdx.x;
    int block = blockIdx.x;

    // Границы подмассивов
    int left  = block * 2 * width;
    int mid   = min(left + width, n);
    int right = min(left + 2 * width, n);

    // Загрузка данных в shared memory
    for (int i = left + tid; i < right; i += blockDim.x)
        s[i - left] = input[i];

    __syncthreads();

    int i = 0;               // Левый подмассив
    int j = mid - left;      // Правый подмассив
    int k = left;            // Индекс записи результата

    // Слияние двух отсортированных подмассивов
    while (i < mid - left && j < right - left)
        output[k++] = (s[i] < s[j]) ? s[i++] : s[j++];

    // Копирование оставшихся элементов
    while (i < mid - left)   output[k++] = s[i++];
    while (j < right - left) output[k++] = s[j++];
}

// =====================================================
// MAIN
// =====================================================
int main() {
    // Размеры массивов для тестирования
    std::vector<int> sizes = {10000, 100000, 1000000};

    for (int n : sizes) {
        std::cout << "\nРазмер массива: " << n << std::endl;

        // ---------------- Подготовка данных ----------------
        std::vector<int> h_data(n);
        generateArray(h_data);

        int* d_data;
        cudaMalloc(&d_data, n * sizeof(int));
        cudaMemcpy(d_data, h_data.data(),
                   n * sizeof(int), cudaMemcpyHostToDevice);

        // ================= REDUCTION =================
        int* d_sum;
        cudaMalloc(&d_sum, sizeof(int));

        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        // ---- Редукция: только глобальная память ----
        cudaMemset(d_sum, 0, sizeof(int));
        cudaEventRecord(start);
        reductionGlobal<<<(n + THREADS - 1) / THREADS, THREADS>>>(d_data, d_sum, n);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        float tGlobal;
        cudaEventElapsedTime(&tGlobal, start, stop);
        std::cout << "Reduction (global): " << tGlobal << " ms\n";

        // ---- Редукция: глобальная + разделяемая память ----
        cudaMemset(d_sum, 0, sizeof(int));
        cudaEventRecord(start);
        reductionShared<<<(n + THREADS - 1) / THREADS, THREADS>>>(d_data, d_sum, n);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        float tShared;
        cudaEventElapsedTime(&tShared, start, stop);
        std::cout << "Reduction (shared): " << tShared << " ms\n";

        // ================= SORT =================
        int chunk  = 256;          // Размер подмассива
        int blocks = n / chunk;    // Количество блоков

        int* d_temp;
        cudaMalloc(&d_temp, n * sizeof(int));

        cudaEventRecord(start);

        // Сортировка подмассивов пузырьком
        bubbleSubarray<<<blocks, 1>>>(d_data, chunk);

        // Поэтапное слияние подмассивов
        for (int w = chunk; w < n; w *= 2) {
            mergeShared<<<(n + 2*w - 1) / (2*w),
                           THREADS,
                           2*w * sizeof(int)>>>(d_data, d_temp, w, n);
            std::swap(d_data, d_temp);
        }

        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        float tSort;
        cudaEventElapsedTime(&tSort, start, stop);
        std::cout << "Sort (optimized): " << tSort << " ms\n";

        // Освобождение памяти GPU
        cudaFree(d_data);
        cudaFree(d_temp);
        cudaFree(d_sum);
    }

    return 0;
}


Overwriting gpu_memory_optimization.cu


In [4]:
!nvcc gpu_memory_optimization.cu -o gpu_mem
!./gpu_mem


Размер массива: 10000
Reduction (global): 7.43219 ms
Reduction (shared): 0.004224 ms
Sort (optimized): 0.006144 ms

Размер массива: 100000
Reduction (global): 0.002048 ms
Reduction (shared): 0.002624 ms
Sort (optimized): 0.004992 ms

Размер массива: 1000000
Reduction (global): 0.007296 ms
Reduction (shared): 0.002688 ms
Sort (optimized): 0.006144 ms
